In [1]:
!git clone https://github.com/yossigandelsman/rosetta_neurons.git

Cloning into 'rosetta_neurons'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 152 (delta 42), reused 99 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 35.27 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
cd rosetta_neurons

/content/rosetta_neurons


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

#@title Requirements
!pip install -r requirements.txt
!pip install transformers
!pip install openai-clip
!pip install einops
!pip install pytorch_pretrained_biggan
!pip install Ninja
!pip install timm==0.4.12
!pip install dill

  Cloning https://github.com/openai/CLIP.git (to revision b46f5ac7587d2e1862f8b7b1573179d80dcdd620) to /tmp/pip-install-jxjdz16b/clip_afc86a82a8b442dfb01d3b62accc86c9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-jxjdz16b/clip_afc86a82a8b442dfb01d3b62accc86c9
  Running command git rev-parse -q --verify 'sha^b46f5ac7587d2e1862f8b7b1573179d80dcdd620'
  Running command git fetch -q https://github.com/openai/CLIP.git b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Running command git checkout -q b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/arogozhnikov/einops.git (to revision d6f7910dc80c8413b7bb400f6f6ad24e3c5f5b0f) to /tmp/pip-install-jxjdz16b/einops_ec76ffeba89c465897c7423ef5d88ee4
  Running command git clone --filter=blob:none --quiet https://github.com/arogozhnikov/eino

In [ ]:
#@title Imports
from transformers import CLIPProcessor, CLIPModel
import torch
import torch.hub
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import math
from functools import reduce
from match_utils import matching, stats,layers,proggan, nethook, dataset, loading, plotting, layers, models, visualize_rosetta, universal
device = torch.device('cuda')

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [6]:
# #@title You can use tables for class 226 used in the paper
# !mkdir "/content/gdrive/MyDrive/results_226_dino_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1w4JS3TVlxQozMBUePIB0datXIoOA_gAh" -O "/content/gdrive/MyDrive/results_226_dino_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1x3YPe-v9pZtY3YUoYxcdDhlSz-q5nvGJ" -O "/content/gdrive/MyDrive/results_226_dino_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1ivJNZmBZuuMCAuPT0s22pLoPdtBx6q9t" -O "/content/gdrive/MyDrive/results_226_dino_styleganxl/gan_stats.pkl"

# !mkdir "/content/gdrive/MyDrive/results_226_clip_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1gGV2_IwByYZ_YbGf9Bc54D3Y3HoN2Tw6" -O "/content/gdrive/MyDrive/results_226_clip_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1POMpSmq5qgpoMk5LOodid8AZRc-VkBRX" -O "/content/gdrive/MyDrive/results_226_clip_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1e_hjbrr6gCIEWFYM-sYjklkM6jagQXb4" -O "/content/gdrive/MyDrive/results_226_clip_styleganxl/gan_stats.pkl"

# !mkdir "/content/gdrive/MyDrive/results_226_resnet50_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1jo9QUPrJBqyMQfr2KehWdCYVJNTE3fa4" -O "/content/gdrive/MyDrive/results_226_resnet50_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1rKLbaq4woSIxTjxrp1ttgXg84JgQoI_u" -O "/content/gdrive/MyDrive/results_226_resnet50_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1sGR1WT8lD28UzdYL0zVCEzn99yOn_Bxr" -O "/content/gdrive/MyDrive/results_226_resnet50_styleganxl/gan_stats.pkl"

# !mkdir "/content/gdrive/MyDrive/results_226_mae_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1u48oWwiLSRQ0H4NojCUeLHFjju-dVUxp" -O "/content/gdrive/MyDrive/results_226_mae_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=15Pkum2cyTQNkTLMCIGxZE4bBOaYIFG-J" -O "/content/gdrive/MyDrive/results_226_mae_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1C18mQ9PCesizO77ekryfVJWhtjuoLnht" -O "/content/gdrive/MyDrive/results_226_mae_styleganxl/gan_stats.pkl"

# !mkdir "/content/gdrive/MyDrive/results_226_dinovitb16_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1aFCKkHk8EJx3IsHe3xz2ete8Evwmr8Bk" -O "/content/gdrive/MyDrive/results_226_dinovitb16_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1A4-tkfsBtRRiYM2FXOkMFy-j7no-JAZZ" -O "/content/gdrive/MyDrive/results_226_dinovitb16_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1FyYJAg8gRkSHtJUodtkAkfVapYyPlG8H" -O "/content/gdrive/MyDrive/results_226_dinovitb16_styleganxl/gan_stats.pkl"




mkdir: cannot create directory ‘/content/gdrive/MyDrive/results_226_dino_styleganxl’: No such file or directory
Error:

	[Errno 2] No such file or directory:
	'/content/gdrive/MyDrive/results_226_dino_styleganxl'

To report issues, please visit https://github.com/wkentaro/gdown/issues.
Error:

	[Errno 2] No such file or directory:
	'/content/gdrive/MyDrive/results_226_dino_styleganxl'

To report issues, please visit https://github.com/wkentaro/gdown/issues.
Error:

	[Errno 2] No such file or directory:
	'/content/gdrive/MyDrive/results_226_dino_styleganxl'

To report issues, please visit https://github.com/wkentaro/gdown/issues.
mkdir: cannot create directory ‘/content/gdrive/MyDrive/results_226_clip_styleganxl’: No such file or directory
Error:

	[Errno 2] No such file or directory:
	'/content/gdrive/MyDrive/results_226_clip_styleganxl'

To report issues, please visit https://github.com/wkentaro/gdown/issues.
Error:

	[Errno 2] No such file or directory:
	'/content/gdrive/MyDrive/resu

In [6]:
#@title You can use tables for class 226 used in the paper
# define directories in Python so we can interpolate them in shell commands
ROOT = "/content/drive/MyDrive/College/vision_research/code/rosetta_analysis"
MATCHES_DIR = f"{ROOT}/matches"

# 1) Dino + StyleGANXL
BASE_DIR = f"{MATCHES_DIR}/results_226_dino_styleganxl"
!mkdir -p {BASE_DIR}
!gdown "https://drive.google.com/uc?export=download&id=1w4JS3TVlxQozMBUePIB0datXIoOA_gAh"   -O {BASE_DIR}/table.pkl
!gdown "https://drive.google.com/uc?export=download&id=1x3YPe-v9pZtY3YUoYxcdDhlSz-q5nvGJ"   -O {BASE_DIR}/discr_stats.pkl
!gdown "https://drive.google.com/uc?export=download&id=1ivJNZmBZuuMCAuPT0s22pLoPdtBx6q9t"   -O {BASE_DIR}/gan_stats.pkl

# 2) CLIP + StyleGANXL
BASE_DIR = f"{MATCHES_DIR}/results_226_clip_styleganxl"
!mkdir -p {BASE_DIR}
!gdown "https://drive.google.com/uc?export=download&id=1gGV2_IwByYZ_YbGf9Bc54D3Y3HoN2Tw6"   -O {BASE_DIR}/table.pkl
!gdown "https://drive.google.com/uc?export=download&id=1POMpSmq5qgpoMk5LOodid8AZRc-VkBRX"   -O {BASE_DIR}/discr_stats.pkl
!gdown "https://drive.google.com/uc?export=download&id=1e_hjbrr6gCIEWFYM-sYjklkM6jagQXb4"   -O {BASE_DIR}/gan_stats.pkl

# 3) ResNet50 + StyleGANXL
BASE_DIR = f"{MATCHES_DIR}/results_226_resnet50_styleganxl"
!mkdir -p {BASE_DIR}
!gdown "https://drive.google.com/uc?export=download&id=1jo9QUPrJBqyMQfr2KehWdCYVJNTE3fa4"   -O {BASE_DIR}/table.pkl
!gdown "https://drive.google.com/uc?export=download&id=1rKLbaq4woSIxTjxrp1ttgXg84JgQoI_u"   -O {BASE_DIR}/discr_stats.pkl
!gdown "https://drive.google.com/uc?export=download&id=1sGR1WT8lD28UzdYL0zVCEzn99yOn_Bxr"   -O {BASE_DIR}/gan_stats.pkl

# 4) MAE + StyleGANXL
BASE_DIR = f"{MATCHES_DIR}/results_226_mae_styleganxl"
!mkdir -p {BASE_DIR}
!gdown "https://drive.google.com/uc?export=download&id=1u48oWwiLSRQ0H4NojCUeLHFjju-dVUxp"   -O {BASE_DIR}/table.pkl
!gdown "https://drive.google.com/uc?export=download&id=15Pkum2cyTQNkTLMCIGxZE4bBOaYIFG-J"   -O {BASE_DIR}/discr_stats.pkl
!gdown "https://drive.google.com/uc?export=download&id=1C18mQ9PCesizO77ekryfVJWhtjuoLnht"   -O {BASE_DIR}/gan_stats.pkl

# 5) DINO ViT-B16 + StyleGANXL
BASE_DIR = f"{MATCHES_DIR}/results_226_dinovitb16_styleganxl"
!mkdir -p {BASE_DIR}
!gdown "https://drive.google.com/uc?export=download&id=1aFCKkHk8EJx3IsHe3xz2ete8Evwmr8Bk"   -O {BASE_DIR}/table.pkl
!gdown "https://drive.google.com/uc?export=download&id=1A4-tkfsBtRRiYM2FXOkMFy-j7no-JAZZ"   -O {BASE_DIR}/discr_stats.pkl
!gdown "https://drive.google.com/uc?export=download&id=1FyYJAg8gRkSHtJUodtkAkfVapYyPlG8H"   -O {BASE_DIR}/gan_stats.pkl


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1w4JS3TVlxQozMBUePIB0datXIoOA_gAh
From (redirected): https://drive.google.com/uc?export=download&id=1w4JS3TVlxQozMBUePIB0datXIoOA_gAh&confirm=t&uuid=8770c249-8a08-48c1-9d98-1d640b277e05
To: /content/drive/MyDrive/College/vision_research/code/rosetta_analysis/matches/results_226_dino_styleganxl/table.pkl
100% 406M/406M [00:05<00:00, 70.3MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1x3YPe-v9pZtY3YUoYxcdDhlSz-q5nvGJ
To: /content/drive/MyDrive/College/vision_research/code/rosetta_analysis/matches/results_226_dino_styleganxl/discr_stats.pkl
100% 48.7k/48.7k [00:00<00:00, 60.7MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1ivJNZmBZuuMCAuPT0s22pLoPdtBx6q9t
To: /content/drive/MyDrive/College/vision_research/code/rosetta_analysis/matches/results_226_dino_styleganxl/gan_stats.pkl
100% 287k/287k [00:00<00:00, 6.40MB/s]
Downloading...
From (original): https://drive

In [7]:
#@title Obtain Rosetta Neurons

resnet50_table_path = "/College/vision_research/code/rosetta_analysis/matches/results_226_resnet50_styleganxl"  #@param {type:"string"}
resnet50_table_path = "/content/drive/MyDrive/"+resnet50_table_path

mae_table_path = "/College/vision_research/code/rosetta_analysis/matches/results_226_mae_styleganxl"  #@param {type:"string"}
mae_table_path = "/content/drive/MyDrive/"+mae_table_path

dino_table_path = "/College/vision_research/code/rosetta_analysis/matches/results_226_dino_styleganxl"  #@param {type:"string"}
dino_table_path = "/content/drive/MyDrive/"+dino_table_path

dinovitb16_table_path = "/College/vision_research/code/rosetta_analysis/matches/results_226_dinovitb16_styleganxl"  #@param {type:"string"}
dinovitb16_table_path = "/content/drive/MyDrive/"+dinovitb16_table_path

clip_table_path = "/College/vision_research/code/rosetta_analysis/matches/results_226_clip_styleganxl"  #@param {type:"string"}
clip_table_path = "/content/drive/MyDrive/"+clip_table_path

n  = 5 #@param {type:"number"}

# calculate Rosetta Neurons
universal_matches, stats = universal.get_universal_activations(resnet50_table_path,
                          mae_table_path,
                          dino_table_path,
                          dinovitb16_table_path,
                          clip_table_path,
                           device = device, n = n)



Loading...
Done
Loading...
Done
Loading...
Done
Loading...
Done
Loading...
Done


In [18]:
len(stats[0][1])

4

In [19]:
#@title Select Parameters
gan_mode = 'styleganxl' #@param ['biggan','styleganxl','stylegan2-lsun_cat', 'stylegan2-lsun_horse', 'stylegan2-lsun_car','stylegan3', 'projgan']


if "stylegan2" in gan_mode or gan_mode == "stylegan3" or gan_mode == "projgan":
  classidx  = None
else:
  classidx  = 226 #@param {type:"raw"}

if gan_mode == "stylegan3-afhq":
  !gdown "https://drive.google.com/uc?export=download&id=1TWqhCvcAh2Qcu8XQMDoCKwGkqfOFpqL2"
elif gan_mode == "stylegan2-lsun_car":
  !gdown "https://drive.google.com/uc?export=download&id=1kEZG4_-hFVwEYyYwfmXGbJFu9KFSN53s"
elif gan_mode == "stylegan2-lsun_cat":
  !gdown "https://drive.google.com/uc?export=download&id=1nB5RQPrWRfcLI7yHhKKkf4gjxrJW7HCj"
elif gan_mode == "stylegan2-lsun_horse":
  !gdown "https://drive.google.com/uc?export=download&id=16SHYO836iF1AGgMps-EFXP0TxDaXYp3g"

!gdown "https://drive.google.com/uc?export=download&id=1uJJHiQ6v8Kec9lpG0RXiyeszfGEMOW-6"


###############################################################
## Get Layers
gan, gan_layers = models.load_gan(gan_mode, device)

resnet50, resnet50_layers = models.load_discr("resnet50", device)
mae, mae_layers = models.load_discr("mae", device)
dino, dino_layers = models.load_discr("dino", device)
dinovit, dinovit_layers = models.load_discr("dino_vitb16", device)
clip, clip_layers = models.load_discr("clip", device)

ganlayers, resnet50layers = layers.get_layers(gan,gan_layers, resnet50, resnet50_layers,"styleganxl", "resnet50", device)
_, maelayers = layers.get_layers(gan,gan_layers, mae, mae_layers,"styleganxl", "mae", device)
_, dinolayers = layers.get_layers(gan,gan_layers, dino, dino_layers,"styleganxl", "dino", device)
_, dinovitlayers = layers.get_layers(gan,gan_layers, dinovit, dinovit_layers,"styleganxl", "dino_vitb16", device)
_, cliplayers = layers.get_layers(gan,gan_layers, clip, clip_layers,"styleganxl", "clip", device)


#create dataset
z_dataset, c_dataset = dataset.create_dataset(gan, gan_mode, 1, 1, classidx, device)




#visualize
visualize_rosetta.viz_matches(gan,\
                            [resnet50, mae, dino, dinovit, clip],\
                            (z_dataset,c_dataset),\
                            ganlayers, \
                            [resnet50layers, maelayers, dinolayers, dinovitlayers, cliplayers],\
                            [stats[0][0], stats[1][0], stats[2][0], stats[3][0], stats[4][0]],\
                            [stats[0][1], stats[1][1], stats[2][1], stats[3][1], stats[4][1]],\
                            "styleganxl", ["resnet50", "mae", "dino", "dino_vitb16", "clip"],\
                            universal_matches)




Output hidden; open in https://colab.research.google.com to view.